In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split # to split the dataset for training and testing
import random
from sklearn.metrics import precision_recall_fscore_support as score
import math

Using TensorFlow backend.


In [2]:
df = pd.read_csv('C:/Users/HI/Documents/internship recommendation system/ml-25m/ratings.csv')
movies = pd.read_csv('C:/Users/HI/Documents/internship recommendation system/ml-25m/movies.csv')

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
new_df = pd.concat([movies.drop('genres', 1), movies['genres'].str.get_dummies(sep="|")], 1)

In [6]:
df = pd.merge(df, new_df, on='movieId')

In [7]:
def f(user):
    profile_1 = df[df['userId']==user]
    profile_1.drop(['movieId','title','timestamp','userId'], axis=1, inplace=True)
    profile_1["sum"] = profile_1.sum(axis=1)
    profile1_new = profile_1.loc[:,"rating":"Western"].div(profile_1["sum"], axis=0)
    return profile1_new

In [8]:
user = pd.DataFrame()
def fitness_func(weights):
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    w5 = weights[4]
    w6 = weights[5]
    w7 = weights[6]
    w8 = weights[7]
    w9 = weights[8]
    w10 = weights[9]
    w11 = weights[10]
    w12 = weights[11]
    w13 = weights[12]
    w14 = weights[13]
    w15 = weights[14]
    w16 = weights[15]
    w17 = weights[16]
    w18 = weights[17]
    w19 = weights[18]
    w20 = weights[19]
    
    w_sum = w1+w2+w3+w4+w5+w6+w7+w8+w9+w10+w11+w12+w13+w14+w15+w16+w17+w18+w19+w20
    
    w1 = w1/w_sum
    w2 = w2/w_sum
    w3 = w3/w_sum
    w4 = w4/w_sum
    w5 = w5/w_sum
    w6 = w6/w_sum
    w7 = w7/w_sum
    w8 = w8/w_sum
    w9 = w9/w_sum
    w10 = w10/w_sum
    w11 = w11/w_sum
    w12 = w12/w_sum
    w13 = w13/w_sum
    w14 = w14/w_sum
    w15 = w15/w_sum
    w16 = w16/w_sum
    w17 = w17/w_sum
    w18 = w18/w_sum
    w19 = w19/w_sum
    w20 = w20/w_sum

    ans = 0
    for index, row in user.iterrows():
        ans += (w1*row.rating + w2*row.Action + w3*row.Adventure + w4*row.Animation + w5*row.Children + w6*row.Comedy + w7*row.Crime + w8*row.Documentary + w9*row.Drama + w10*row.Fantasy + w11*SciFi + w12*row.Horror + w13*row.IMAX + w14*row.Musical + w15*row.Mystery + w16*row.Romance + w17*FilmNoir * w18*row.Thriller + w19*row.War + w20*row.Western)**2

    return ans

In [9]:
lb = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
ub = [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255]

In [10]:
# implementation of PSO
# checking and defining weight inside the function; a random number between 0.5 and 1
def pso(fitness_func, lbound, ubound, swarm_size=75, max_iter=300, dimensions=20, c1=1.494, c2=1.494):
    x=[]            # positon of each particle
    x_pbest=[]      # best position of each particle 
    x_gbest=[]      # best global position of population
    v=[]            # velocity of each particle
    v_min=[]
    v_max=[]
    
    # assign initial random positions to the particles
    for i in range(swarm_size):
        x.append([])
        x_pbest.append([])
        for j in range(dimensions):
            x[i].append(random.uniform(lbound[j],ubound[j]))
            x_pbest[i].append(x[i][j])
            
    #calculate initial group best of the population
    for i in range(swarm_size):
        if i==0:
            x_gbest=x[i]
        elif fitness_func(x[i])<fitness_func(x_gbest):
            x_gbest=x[i]

    
    # calculate minimum and maximum boundaries of velocity vector
    for i in range(dimensions):
        v_min.append(-(ubound[i]-lbound[i])/100)
        v_max.append((ubound[i]-lbound[i])/100)
 
    # assign initial random velocities to the particles
    for i in range(swarm_size):
        v.append([])
        for j in range(dimensions):
            v[i].append(random.uniform(v_min[j],v_max[j]))
            
    for iter in range(max_iter):
        for i in range(swarm_size):
            for j in range(dimensions):
                r1=random.uniform(0,1)
                r2=random.uniform(0,1)
                weight = random.uniform(0.5, 1)
                # calculate new velocity for each particle
                v[i][j] = weight*(v[i][j]) + r1*c1*(x_pbest[i][j]-x[i][j]) + r2*c2*(x_gbest[j]-x[i][j])
                
                if v[i][j] > v_max[j]:
                    v[i][j] = v_max[j]
                
                if v[i][j] < v_min[j]:
                    v[i][j] = v_min[j]
                                 
                # calculate new position for each particle
                x[i][j] = x[i][j] + v[i][j]
                
                if x[i][j] > ubound[j]:
                    x[i][j] = ubound[j]
                    
                if x[i][j] < lbound[j]:
                    x[i][j] = lbound[j]
            
            if fitness_func(x[i]) < fitness_func(x_pbest[i]):
                x_pbest[i] = x[i]
                
            if fitness_func(x[i]) < fitness_func(x_gbest):
                x_gbest = x[i]
                
    return x_gbest,fitness_func(x_gbest)

In [11]:
# function to predict weights of a particular user using Particle Swarm Optimization
def predict_weights_PSO(df, user):
    user_df = df[df['userId']==user]
    # estimating weights using pso
    xopt, fopt = pso(fitness_func, lb, ub, swarm_size=75,max_iter=300)
    return xopt

In [12]:
def predict_weights_all_pso(df):
    user_weight_dict = {}
    i=0
    for user in df.userId.unique():
        i+=1
        xopt = predict_weights_PSO(df, user)
        user_weight_dict[user] = xopt
        print(i)
    return user_weight_dict

user_weight_dict = predict_weights_all_pso(df)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [22]:
user_weight_dict

{1: [237.24474230891008,
  100.88359879452157,
  202.94655617561085,
  89.09302788823926,
  193.26915182456597,
  13.8500329986272,
  206.3742246937724,
  209.40729265653147,
  239.11738544569198,
  247.701547359038,
  158.82471603289605,
  179.99481538619906,
  253.04791720043684,
  148.863782227007,
  161.25644791250272,
  75.37633423876596,
  28.93721891171887,
  85.69602654721288,
  104.105565250855,
  87.69550797189387],
 5: [208.76430809207494,
  92.2324628664921,
  208.88003982206857,
  114.29661767630873,
  63.61333735244927,
  172.43802250834972,
  95.74507409915613,
  113.77609510902599,
  251.8065967518446,
  244.27531428528147,
  198.6964285498719,
  231.26598278428293,
  40.094232562349106,
  65.39026881339966,
  182.94112720501963,
  148.58450427721138,
  88.45720545201334,
  182.12405135387164,
  163.09718603203,
  70.28055296598012],
 7: [160.70799112852637,
  92.08089024441217,
  90.01389089721332,
  221.63738797871162,
  214.37838883966756,
  139.84968517488917,
  35.

In [13]:
# functions for finding similarity between two users based on their preferences to individual criteria
# based on euclidean distance
# where weights is a list of individual weights of user1
def euclidean_similarity_weighted(df, user1, user2, weights):
    s = pd.merge(df[df['userId']==user1], df[df['userId']==user2], how="inner", on="movieId")
    sum1 = 0
    similarity=0
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    w5 = weights[4]
    w6 = weights[5]
    w7 = weights[6]
    w8 = weights[7]
    w9 = weights[8]
    w10 = weights[9]
    w11 = weights[10]
    w12 = weights[11]
    w13 = weights[12]
    w14 = weights[13]
    w15 = weights[14]
    w16 = weights[15]
    w17 = weights[16]
    w18 = weights[17]
    w19 = weights[18]
    w20 = weights[19]
    
    for index, row in s.iterrows():
           
        tempvar = (w1*((row.rating_x - row.rating_y)**2) + \
                w2*((row.Action_x - row.Action_y)**2) + \
                w3*((row.Adventure_x - row.Adventure_y)**2) + \
                w4*((row.Animation_x - row.Animation_y)**2) + \
                w5*((row.Children_x - row.Children_y)**2) + \
                w6*((row.Comedy_x - row.Comedy_y)**2) + \
                w7*((row.Crime_x - row.Crime_y)**2) + \
                w8*((row.Documentary_x - row.Documentary_y)**2) + \
                w9*((row.Drama_x - row.Drama_y)**2) + \
                w10*((row.Fantasy_x - row.Fantasy_y)**2) + \
                w11*((row.SciFi_x-row.SciFi_y)**2) + \
                w12*((row.Horror_x - row.Horror_y)**2) + \
                w13*((row.IMAX_x - row.IMAX_y)**2) + \
                w14*((row.Musical_x - row.Musical_y)**2) + \
                w15*((row.Mystery_x - row.Mystery_y)**2) + \
                w16*((row.Romance_x - row.Romance_y)**2) + \
                w17*((row.FilmNoir_x-row.FilmNoir_y)**2) + \
                w18*((row.Thriller_x - row.Thriller_y)**2) + \
                w19*((row.War_x - row.War_y)**2) + \
                w20*((row.Western_x - row.Western_y)**2)    
                  ) 
        tempvar = max(0, tempvar)
        tempvar = math.sqrt(tempvar)
        sum1 += tempvar
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity

In [14]:
def similarUsers(user):
    k = 25
    sim_scores = []
    for i in df.userId.unique():
        s = euclidean_similarity_weighted(df, user, i, user_weight_dict[user])
        #s = correlation(user,i)
        #s = euclidean_similarity(user,i)
        sim_scores.append(s)
    simusers = {'users':df.userId.unique(), 'similarity_score':sim_scores}
    simusers = pd.DataFrame(simusers)
    simusers = simusers.nlargest(k, 'similarity_score')
    return simusers

In [15]:
def predictRating(user,movie):
    n = 0
    predRating = 0
    k = 25
    #s = similarUsers(user)
    for i in similar_user['users']:
        rating = df[(df['userId']==i) & (df['movieId'] == movie)]['rating'].values
        sim = similar_user[similar_user['users']==i]['similarity_score'].values
        if(len(rating)!=0):
            predRating = predRating + (rating * sim)
    predRating = predRating/k 
    rating = df[df['userId']==user]['rating'].values
    average = np.average(rating)
    predRating = average+predRating
    if(predRating>5):
        predRating = 5
    return predRating

In [16]:
def getTopMovies(user):
    top_movies = []
    #s = similarUsers(testUser)
    for i in similar_user['users']:
        if i == user:
            continue
        movies = df[(df['userId']==i) & (df['rating']>4)]['movieId'].values
        for j in movies:
            if j not in df[df['userId']==user]['movieId'].values:
                if j not in top_movies:
                    top_movies.append(j)
    return top_movies

In [17]:
def getTopMovies1(user):
    top_movies = []
    #s = similarUsers(testUser)
    for i in similar_user['users']:
        if i == user:
            continue
        movies = df[(df['userId']==i) & (df['rating']>4)]['movieId'].values
        for j in movies:
            if j not in top_movies:
                top_movies.append(j)
    return top_movies

In [24]:
def recommendation(testUser):
    mo = []
    p = []
    u = []
    m = getTopMovies(testUser)
    pr = 50
    if(len(m)<50):
        pr = len(m)
    for j in range(0,pr):
        predRating = predictRating(testUser,m[j])
        mo.append(m[j])
        u.append(testUser)
        #print(j)
        if np.isscalar(predRating):
            p.append(predRating)
        else:
            p.append(predRating[0])
    d = {'userId':u,'movieId':mo, 'predicted rating': p}
    df_u = pd.DataFrame(d)
    df_u = df_u.nlargest(50, 'predicted rating') 
    return df_u

In [25]:
def recommendation1(testUser):
    mo = []
    p = []
    u = []
    m = getTopMovies1(testUser)
    pr = 50
    if(len(m)<50):
        pr = len(m)
    for j in range(0,pr):
        predRating = predictRating(testUser,m[j])
        mo.append(m[j])
        u.append(testUser)
        if np.isscalar(predRating):
            p.append(predRating)
        else:
            p.append(predRating[0])
    d = {'userId':u,'movieId':mo, 'predicted rating': p}
    df_u = pd.DataFrame(d)
    df_u = df_u.nlargest(50, 'predicted rating') 
    return df_u

In [26]:
for testUser in range(1,611):
    similar_user = similarUsers(testUser)
    df_u = recommendation(testUser)
    print('recommendations for user', testUser, ':\n')
    l = 10
    if(len(df_u)<10):
        l = len(df_u['movieId'])
    for i in range(0,l):
        m = movies[movies['movieId']==df_u['movieId'][i]]['title'].values
        print(m[0])
    print('\n\n')

recommendations for user 1 :

Dark Knight, The (2008)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Half-Blood Prince (2009)
Avatar (2009)
Ratatouille (2007)
WALL·E (2008)
Avengers, The (2012)
Deadpool (2016)





KeyboardInterrupt: 

In [30]:
df_c = pd.DataFrame(columns=['userId','movieId','predicted rating'])
for testUser in df.userId.unique():
    similar_user = similarUsers(testUser)
    df_u = recommendation1(testUser)
    df_c = pd.concat([df_u, df_c])
    print('recommendations for user', testUser, ':\n')
    l = 10
    if(len(df_u)<10):
        l = len(df_u['movieId'])
    for i in range(0,l):
        m = movies[movies['movieId']==df_u['movieId'][i]]['title'].values
        print(m[0])
    print('\n\n')

recommendations for user 1 :

Dark Knight, The (2008)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Half-Blood Prince (2009)
Avatar (2009)
Ratatouille (2007)
WALL·E (2008)
Avengers, The (2012)
Deadpool (2016)



recommendations for user 5 :

Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Mars Attacks! (1996)
Star Trek: First Contact (1996)
Dark Knight, The (2008)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Half-Blood Prince (2009)
Avatar (2009)
Ratatouille (2007)



recommendations for user 7 :

Seven (a.k.a. Se7en) (1995)
Silence of the Lambs, The (1991)
Dumbo (1941)
Fight Club (1999)
Memento (2000)
28 Days Later (2002)
Evil Dead II (Dead by Dawn) (1987)
Muppet Ch

recommendations for user 76 :

Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Birdcage, The (1996)
Star Trek: First Contact (1996)
Lock, Stock & Two Smoking Barrels (1998)
Departed, The (2006)
Snatch (2000)
Boondock Saints, The (2000)
Layer Cake (2004)
Lucky Number Slevin (2006)
Crash (2004)



recommendations for user 78 :

Requiem for a Dream (2000)
Singin' in the Rain (1952)
Traffic (2000)
Memento (2000)
Beautiful Mind, A (2001)
Life Less Ordinary, A (1997)
Psycho (1960)
Rushmore (1998)
American Beauty (1999)
Kill Bill: Vol. 1 (2003)



recommendations for user 82 :

Lock, Stock & Two Smoking Barrels (1998)
Departed, The (2006)
Snatch (2000)
Boondock Saints, The (2000)
Layer Cake (2004)
Lucky Number Slevin (2006)
Crash (2004)
That Thing You Do! (1996)
Monty Python and the Holy Grail (1975)
Princess Bride, The (1987)



recommendations for user 86 :

Full Metal Jacket (1987)
Shining, The (1980)
Requiem for a Dream (2000)
Memento (2000)
Blade Runner (1982)
Chinatown (1974)
Third Man, The (

recommendations for user 156 :

Dark Knight, The (2008)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Half-Blood Prince (2009)
Avatar (2009)
Ratatouille (2007)
WALL·E (2008)
Avengers, The (2012)
Deadpool (2016)



recommendations for user 159 :

Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Birdcage, The (1996)
Star Trek: First Contact (1996)
Grumpier Old Men (1995)
Twister (1996)
Independence Day (a.k.a. ID4) (1996)
Four Rooms (1995)
Alien (1979)
Starship Troopers (1997)
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)



recommendations for user 160 :

Schindler's List (1993)
Fight Club (1999)
Kingpin (1996)
Me, Myself & Irene (2000)
Usual Suspects, The (1995)
Thin Red Line, The (1998)
Shawshank Redemption, The (1994)
Simpsons Movie, The (2007)
Awakenings (1990)
Big Chill, The (1983)



recommendati

recommendations for user 226 :

Schindler's List (1993)
Dark Knight, The (2008)
Inception (2010)
Bourne Ultimatum, The (2007)
Up (2009)
Fargo (1996)
Big Lebowski, The (1998)
Run Lola Run (Lola rennt) (1998)
Easy Rider (1969)
Good Morning, Vietnam (1987)



recommendations for user 229 :

Fargo (1996)
Leaving Las Vegas (1995)
Seven (a.k.a. Se7en) (1995)
Braveheart (1995)
Pulp Fiction (1994)
Goodfellas (1990)
Indiana Jones and the Last Crusade (1989)
American History X (1998)
Thin Red Line, The (1998)
Office Space (1999)



recommendations for user 232 :

Blues Brothers, The (1980)
Amadeus (1984)
Good, the Bad and the Ugly, The (Buono, il brutto, il cattivo, Il) (1966)
For a Few Dollars More (Per qualche dollaro in più) (1965)
Withnail & I (1987)
Lost Boys, The (1987)
Near Dark (1987)
Hedwig and the Angry Inch (2000)
No Direction Home: Bob Dylan (2005)
Matrix, The (1999)



recommendations for user 233 :

Beetlejuice (1988)
Casino (1995)
Dr. Strangelove or: How I Learned to Stop Worrying

recommendations for user 291 :

Seven (a.k.a. Se7en) (1995)
Usual Suspects, The (1995)
Bottle Rocket (1996)
Rob Roy (1995)
Canadian Bacon (1995)
Desperado (1995)
Billy Madison (1995)
Dumb & Dumber (Dumb and Dumber) (1994)
Star Wars: Episode IV - A New Hope (1977)
Tommy Boy (1995)



recommendations for user 292 :

Usual Suspects, The (1995)
Schindler's List (1993)
Thin Red Line, The (1998)
Shawshank Redemption, The (1994)
Simpsons Movie, The (2007)
Awakenings (1990)
Big Chill, The (1983)
Ice Storm, The (1997)
Platoon (1986)
Last Samurai, The (2003)



recommendations for user 293 :

Star Wars: Episode IV - A New Hope (1977)
Rock, The (1996)
Star Wars: Episode VI - Return of the Jedi (1983)
Rumble in the Bronx (Hont faan kui) (1995)
Chasing Amy (1997)
First Strike (Police Story 4: First Strike) (Ging chaat goo si 4: Ji gaan daan yam mo) (1996)
Private Parts (1997)
Barb Wire (1996)
Twister (1996)
Independence Day (a.k.a. ID4) (1996)



recommendations for user 298 :

Schindler's List (19

...

recommendations for user 487 :

Pulp Fiction (1994)
Schindler's List (1993)
Dances with Wolves (1990)
Pinocchio (1940)
Postman, The (Postino, Il) (1994)
Heavenly Creatures (1994)
In the Name of the Father (1993)
Beauty and the Beast (1991)
Once Were Warriors (1994)
Snow White and the Seven Dwarfs (1937)



recommendations for user 516 :

Pulp Fiction (1994)
Schindler's List (1993)
Dances with Wolves (1990)
Pinocchio (1940)
Postman, The (Postino, Il) (1994)
Heavenly Creatures (1994)
In the Name of the Father (1993)
Beauty and the Beast (1991)
Once Were Warriors (1994)
Snow White and the Seven Dwarfs (1937)



recommendations for user 527 :

Dark Knight, The (2008)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Half-Blood Prince (2009)
Avatar (2009)
Ratatouille (2007)
WALL·E (2008)
Avengers, The (2012)
Deadpool (2016)



recom

recommendations for user 209 :

Toy Story (1995)
Desperado (1995)
Clerks (1994)
Pulp Fiction (1994)
Tombstone (1993)
Shawshank Redemption, The (1994)
Like Water for Chocolate (Como agua para chocolate) (1992)
Babe (1995)
Apollo 13 (1995)
Quiz Show (1994)



recommendations for user 243 :

Star Wars: Episode IV - A New Hope (1977)
Fargo (1996)
Mighty Aphrodite (1995)
Postman, The (Postino, Il) (1994)
Everyone Says I Love You (1996)
Sense and Sensibility (1995)
Leaving Las Vegas (1995)
Very Brady Sequel, A (1996)
Sling Blade (1996)
Shine (1996)



recommendations for user 300 :

Toy Story (1995)
Usual Suspects, The (1995)
Braveheart (1995)
Star Wars: Episode IV - A New Hope (1977)
Pulp Fiction (1994)
Mask, The (1994)
Jurassic Park (1993)
Schindler's List (1993)
Reservoir Dogs (1992)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)



recommendations for user 375 :

Toy Story (1995)
Desperado (1995)
Clerks (1994)
Pulp Fiction (1994)
Tombstone (1993)
Shawshank

recommendations for user 12 :

Star Wars: Episode IV - A New Hope (1977)
Fargo (1996)
Mighty Aphrodite (1995)
Postman, The (Postino, Il) (1994)
Everyone Says I Love You (1996)
Sense and Sensibility (1995)
Leaving Las Vegas (1995)
Very Brady Sequel, A (1996)
Sling Blade (1996)
Shine (1996)



recommendations for user 36 :

Heat (1995)
Seven (a.k.a. Se7en) (1995)
Usual Suspects, The (1995)
Star Wars: Episode IV - A New Hope (1977)
Jurassic Park (1993)
Independence Day (a.k.a. ID4) (1996)
Star Wars: Episode V - The Empire Strikes Back (1980)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Star Wars: Episode VI - Return of the Jedi (1983)
Saving Private Ryan (1998)



recommendations for user 371 :

Usual Suspects, The (1995)
Braveheart (1995)
Pulp Fiction (1994)
Forrest Gump (1994)
Fugitive, The (1993)
Schindler's List (1993)
Silence of the Lambs, The (1991)
Wizard of Oz, The (1939)
Swingers (1996)
Star Wars: Episode VI - Return of the Jedi (1983)



recomme

recommendations for user 236 :

Fargo (1996)
Wizard of Oz, The (1939)
Citizen Kane (1941)
Mr. Smith Goes to Washington (1939)
Monty Python and the Holy Grail (1975)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Apocalypse Now (1979)
Alien (1979)
Psycho (1960)
All Quiet on the Western Front (1930)



recommendations for user 440 :

Toy Story (1995)
Forrest Gump (1994)
Silence of the Lambs, The (1991)
Princess Bride, The (1987)
Alien (1979)
Psycho (1960)
Shining, The (1980)
Shutter Island (2010)
Interstellar (2014)
Rear Window (1954)



recommendations for user 499 :

Braveheart (1995)
Pulp Fiction (1994)
Fargo (1996)
Rock, The (1996)
Monty Python's Life of Brian (1979)
Reservoir Dogs (1992)
Platoon (1986)
Monty Python and the Holy Grail (1975)
Star Wars: Episode V - The Empire Strikes Back (1980)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)



recommendations for user 508 :

Toy Story (1995)
Usual Suspects, The (1995)
Br

recommendations for user 127 :

Shawshank Redemption, The (1994)
Singin' in the Rain (1952)
Wallace & Gromit: A Close Shave (1995)
Wallace & Gromit: The Wrong Trousers (1993)
Grand Day Out with Wallace and Gromit, A (1989)
Creature Comforts (1989)
Bill & Ted's Excellent Adventure (1989)
Chicken Run (2000)
Wallace & Gromit in The Curse of the Were-Rabbit (2005)
Doctor Dolittle (1967)



recommendations for user 184 :

Seven (a.k.a. Se7en) (1995)
Braveheart (1995)
Pulp Fiction (1994)
Forrest Gump (1994)
Dances with Wolves (1990)
Winnie the Pooh and the Blustery Day (1968)
Willy Wonka & the Chocolate Factory (1971)
Monty Python and the Holy Grail (1975)
Clockwork Orange, A (1971)
Full Metal Jacket (1987)



recommendations for user 496 :

Requiem for a Dream (2000)
Singin' in the Rain (1952)
Traffic (2000)
Memento (2000)
Beautiful Mind, A (2001)
Life Less Ordinary, A (1997)
Toy Story (1995)
Office Space (1999)
Shutter Island (2010)
True Lies (1994)



recommendations for user 306 :

Pulp 

In [35]:
df_err = pd.merge(df,df_c,on=['movieId','userId'])
rmse = sqrt(mean_squared_error(df_err['rating'], df_err['predicted rating']))
mse = mean_squared_error(df_err['rating'], df_err['predicted rating'])
print(mse)
print(rmse)

0.55395263348913
0.7442799429577086


In [32]:
df_err

,userId,movieId,rating,timestamp,title,(no genres listed),Action,Adventure,Animation,Children,...,Horror,IMAX,Musical,Mystery,Romance,SciFi,Thriller,War,Western,predicted rating
0,31,1,5.0,850466616,Toy Story (1995),0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,4.520000
1,40,1,5.0,832058959,Toy Story (1995),0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,4.366990
2,43,1,5.0,848993983,Toy Story (1995),0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,5.000000
3,46,1,5.0,834787906,Toy Story (1995),0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,5.000000
4,63,1,5.0,1443199669,Toy Story (1995),0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,3.921629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505,313,3727,5.0,1030475717,Near Dark (1987),0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,3.668169
2506,91,6731,4.5,1112712674,Day of the Dead (1985),0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,3.606281
2507,287,4642,5.0,1110231622,Hedwig and the Angry Inch (2000),0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3.040999
2508,356,4642,5.0,1229139425,Hedwig and the Angry Inch (2000),0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,4.420761


In [33]:
ratings = []
predicted = []
for i in df_err['rating']:
    if(i>3.5):
        ratings.append(1)
    else:
        ratings.append(0)
for i in df_err['predicted rating']:
    if(i>3.5):
        predicted.append(1)
    else:
        predicted.append(0)

In [34]:
from sklearn.metrics import f1_score
f1_score(ratings,predicted)

0.9255649091714665